In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

def scrape_data(url, nombre_columna):
    driver = webdriver.Chrome()
    datos = []
    try:
        driver.get(url)
        espera = WebDriverWait(driver, 10)
        boton_cookie = espera.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cookie-hint-btn-accept"]')))
        boton_cookie.click()
        entrada_fecha_inicio = espera.until(EC.presence_of_element_located((By.XPATH, '//input[@placeholder=""]')))
        entrada_fecha_inicio.clear()
        entrada_fecha_inicio.send_keys("01/01/2020")
        espera.until(EC.presence_of_element_located((By.XPATH, '/html/body/app-root/app-wrapper/div/div[2]/app-equity/div[2]/div/app-price-history/div[2]/div/div/div/table')))

        numero_pagina = 1
        while True:
            try:
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                tabla = soup.find('table')
                filas = tabla.find_all('tr')
                for fila in filas:
                    celdas = fila.find_all('td')
                    if len(celdas) > 2:
                        fecha = celdas[0].text
                        valor = celdas[2].text.replace(',', '.')
                        datos.append([fecha, float(valor)])
                numero_pagina += 1
                boton_siguiente = espera.until(EC.element_to_be_clickable((By.XPATH, f'//button[text()="{numero_pagina}"]')))
                boton_siguiente.click()
            except Exception as e:
                print(f"Todas las páginas exportadas en {nombre_columna}")
                break
    except TimeoutException:
        print(f"No se localizan datos en {url}. Saltamos a la siguiente URL.")
    finally:
        driver.quit()

    df = pd.DataFrame(datos, columns=['Fecha', nombre_columna])
    df[nombre_columna] = df[nombre_columna].round(0).astype(int)
    return df

urls_y_nombres_columnas = [
    ("https://www.boerse-frankfurt.de/equity/adidas-ag/price-history/historical-prices-and-volumes", "Adidas"),
    ("https://www.boerse-frankfurt.de/equity/airbus-se/price-history/historical-prices-and-volumes", "Airbus"),
    ("https://www.boerse-frankfurt.de/equity/allianz-se/price-history/historical-prices-and-volumes", "Allianz"),
    ("https://www.boerse-frankfurt.de/equity/basf-se/price-history/historical-prices-and-volumes", "Basf"),
    ("https://www.boerse-frankfurt.de/equity/bayer-ag/price-history/historical-prices-and-volumes", "Bayer"),
    ("https://www.boerse-frankfurt.de/equity/beiersdorf-aktiengesellschaft/price-history/historical-prices-and-volumes", "Beiersdorf"),
    ("https://www.boerse-frankfurt.de/equity/bmw-ag-st/price-history/historical-prices-and-volumes", "BMW"),
    ("https://www.boerse-frankfurt.de/equity/brenntag-se/price-history/historical-prices-and-volumes", "Brenntag"),
    ("https://www.boerse-frankfurt.de/equity/commerzbank-ag/price-history/historical-prices-and-volumes", "Commerzbank"),
    ("https://www.boerse-frankfurt.de/equity/continental-ag/price-history/historical-prices-and-volumes", "Continental"),
    ("https://www.boerse-frankfurt.de/equity/covestro-ag/price-history/historical-prices-and-volumes", "Covestro"),
    ("https://www.boerse-frankfurt.de/equity/daimler-truck-holding-ag/price-history/historical-prices-and-volumes", "Daimler Truck"),
    ("https://www.boerse-frankfurt.de/equity/deutsche-bank-ag/price-history/historical-prices-and-volumes", "Deutsche Bank"),
    ("https://www.boerse-frankfurt.de/equity/deutsche-boerse-ag/price-history/historical-prices-and-volumes", "Deutsche Borse"),
    ("https://www.boerse-frankfurt.de/equity/deutsche-post-ag/price-history/historical-prices-and-volumes", "Deutsche Post"),
    ("https://www.boerse-frankfurt.de/equity/deutsche-telekom-ag/price-history/historical-prices-and-volumes", "Deutsche Telekom"),
    ("https://www.boerse-frankfurt.de/equity/e-on-se/price-history/historical-prices-and-volumes", "EON"),
    ("https://www.boerse-frankfurt.de/equity/fresenius-se-co-kgaa/price-history/historical-prices-and-volumes", "Fresenius SE"),
    ("https://www.boerse-frankfurt.de/equity/hannover-rueck-se/price-history/historical-prices-and-volumes", "Hannover Ruck"),
    ("https://www.boerse-frankfurt.de/equity/heidelbergcement-ag/price-history/historical-prices-and-volumes", "Heidelberg Materials"),
    ("https://www.boerse-frankfurt.de/equity/henkel-ag-co-kgaa-vz/price-history/historical-prices-and-volumes", "Henkel AG & Co. KGaA Vz"),
    ("https://www.boerse-frankfurt.de/equity/infineon-technologies-ag/price-history/historical-prices-and-volumes", "Infineon Technologies"),
    ("https://www.boerse-frankfurt.de/equity/mercedes-benz-group/price-history/historical-prices-and-volumes", "Mercedez Benz Group"),
    ("https://www.boerse-frankfurt.de/equity/merck-kgaa/price-history/historical-prices-and-volumes", "Merck KGaA"),
    ("https://www.boerse-frankfurt.de/equity/mtu-aero-engines-ag/price-history/historical-prices-and-volumes", "MTU Aero Engines"),
    ("https://www.boerse-frankfurt.de/equity/muenchener-rueckversicherungs-gesellschaft-ag-in-muenchen/price-history/historical-prices-and-volumes", "Muenchener Rueckversicherungs"),
    ("https://www.boerse-frankfurt.de/equity/porsche-ag-vz/price-history/historical-prices-and-volumes", "Porsche AVG"),
    ("https://www.boerse-frankfurt.de/equity/porsche-automobil-holding-se-vz/price-history/historical-prices-and-volumes", "Porsche Automobil Holding SE Vz"),
    ("https://www.boerse-frankfurt.de/equity/qiagen-nv/price-history/historical-prices-and-volumes", "Qiagen"),
    ("https://www.boerse-frankfurt.de/equity/rheinmetall-ag/price-history/historical-prices-and-volumes", "Rheinmetall"),
    ("https://www.boerse-frankfurt.de/equity/rwe-ag-st/price-history/historical-prices-and-volumes", "RWE AG St"),
    ("https://www.boerse-frankfurt.de/equity/sap-se/price-history/historical-prices-and-volumes", "SAP SE"),
    ("https://www.boerse-frankfurt.de/equity/sartorius-ag-vz/price-history/historical-prices-and-volumes", "Sartorius"),
    ("https://www.boerse-frankfurt.de/equity/siemens-ag/price-history/historical-prices-and-volumes", "Siemens AG"),
    ("https://www.boerse-frankfurt.de/equity/siemens-energy-ag/price-history/historical-prices-and-volumes", "Siemens Energy AG"),
    ("https://www.boerse-frankfurt.de/equity/siemens-healthineers-ag/price-history/historical-prices-and-volumes", "Siemens Healthineers"),
    ("https://www.boerse-frankfurt.de/equity/symrise-ag/price-history/historical-prices-and-volumes", "Symrise"),
    ("https://www.boerse-frankfurt.de/equity/volkswagen-ag-vz/price-history/historical-prices-and-volumes", "Volkswagen"),
    ("https://www.boerse-frankfurt.de/equity/vonovia-se/price-history/historical-prices-and-volumes", "Vonovia"),
    ("https://www.boerse-frankfurt.de/equity/zalando-se/price-history/historical-prices-and-volumes", "Zalando SE"),
]

Dax40 = None

for url, nombre_columna in urls_y_nombres_columnas:
    df = scrape_data(url, nombre_columna)
    if Dax40 is None:
        Dax40 = df
    else:
        Dax40 = pd.merge(Dax40, df, on='Fecha', how='outer')

Dax40['Fecha'] = pd.to_datetime(Dax40['Fecha'], format='%d/%m/%y').dt.date
Dax40['Fecha'] = Dax40['Fecha'].astype(str)
Dax40 = Dax40.sort_values('Fecha', ascending=False)
Dax40 = Dax40.reset_index(drop=True)

valores_moda = []

for columna in Dax40.columns[1:]:
    serie_moda = Dax40[columna].dropna().mode()
    if not serie_moda.empty:
        valor_moda = serie_moda[0]
        valores_moda.append([columna, valor_moda])
    else:
        valores_moda.append([columna, None])

df_moda = pd.DataFrame(valores_moda, columns=['Columna', 'Valor más repetido'])
print(df_moda)

Todas las páginas exportadas en Adidas
Todas las páginas exportadas en Airbus
Todas las páginas exportadas en Allianz
Todas las páginas exportadas en Basf
Todas las páginas exportadas en Bayer
Todas las páginas exportadas en Beiersdorf
Todas las páginas exportadas en BMW
Todas las páginas exportadas en Brenntag
Todas las páginas exportadas en Commerzbank
Todas las páginas exportadas en Continental
Todas las páginas exportadas en Covestro
Todas las páginas exportadas en Daimler Truck
Todas las páginas exportadas en Deutsche Bank
Todas las páginas exportadas en Deutsche Borse
Todas las páginas exportadas en Deutsche Post
Todas las páginas exportadas en Deutsche Telekom
Todas las páginas exportadas en EON
Todas las páginas exportadas en Fresenius SE
Todas las páginas exportadas en Hannover Ruck
Todas las páginas exportadas en Heidelberg Materials
Todas las páginas exportadas en Henkel AG & Co. KGaA Vz
Todas las páginas exportadas en Infineon Technologies
Todas las páginas exportadas en Me